<a href="https://colab.research.google.com/github/peefeeyatko/decision-tree-classifier/blob/main/DecisionTreeClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Implementation of CART Decision Tree Algorithm**

The problem this Decision Tree algorithm solves involves predicting whether a banknote is authentic or not. 

The dataset provides a number of samples of banknotes and their measurements taken from a very detailed photograph.

Each data sample is categorized into one of two classifications: 1 (authentic) or 0 (inauthentic), making this a binary classification problem.

### **Initial setup and data preprocessing**

In [ ]:
import enum
import random
from urllib.request import urlopen

# get the raw data in csv format
data = urlopen('https://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt')

# initial list to store each sample
dataset = []

# iterate over the raw data and append each sample to the dataset
for line in data:
    sample = line.decode('utf-8')
    sample = sample.split(',')

    for i in range(len(sample)):
      if i < 4:
        # cast attributes/features as floats
        sample[i] = float(sample[i])
      else:
        # strip class label of any trailing characters
        sample[i] = sample[i].rstrip()
    
    # append preprocessed sample to dataset
    dataset.append(sample)

### **Explore the dataset**

In [ ]:
# attribute names / column headers
attr_names = ["variance", "skewness", "curtosis", "entropy", "label"]

# print out the first and last data samples
print(dataset[0])
print(dataset[-1])

[3.6216, 8.6661, -2.8073, -0.44699, '0']
[-2.5419, -0.65804, 2.6842, 1.1952, '1']


### **Partition data into training and testing sets**

In [ ]:
# partition the dataset into training and testing sets
training = []
testing = []

# shuffle the dataset so the label distribution is more mixed
random.shuffle(dataset)

# split 80% into training and remainder into testing
split = round(len(dataset) * 0.8)
training = dataset[0:split]
testing = dataset[split:]

print("Number of training samples: ", len(training))
print("Number of testing samples: ", len(testing))

Number of training samples:  1098
Number of testing samples:  274


### **Algorithm implementation**

In [ ]:
# use a enum to reference full classification name
class Classification(enum.Enum):
    inauthentic = '0'
    authentic = '1'

In [ ]:
def class_counts(rows):
    """Returns a dictionary with distinct labels and their count."""
    counts = {}
    for row in rows:
        # in the banknote dataset the label is the last column
        label = Classification(row[-1]).name
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [ ]:
# display the amount of each class/label in the training data
class_counts(training)

{'authentic': 479, 'inauthentic': 619}

In [ ]:
class Question:
    """A Question is used to partition the dataset."""

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if isinstance(val, int) or isinstance(val, float):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
      # format the question for printing in the display_tree function
      return '{0} >= {1}'.format(attr_names[self.column], self.value)

In [ ]:
def partition_at_node(rows, question):
    """Partition and return two subsets, 
    the true rows and false rows based on the question"""
    true_rows = [] 
    false_rows = []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [ ]:
def gini_impurity(rows):
    """Subtracting the sum of squared probabilites from 1"""
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl ** 2
    return impurity

In [ ]:
def information_gain(left, right, current_uncertainty):
    """Calculates a weighted average of the uncertainties
    and then subtracts from the current uncertainty"""
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini_impurity(left) - (1 - p) * gini_impurity(right)

In [ ]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0
    best_question = None
    current_uncertainty = gini_impurity(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:
            # ask a question
            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition_at_node(rows, question)

            # skip this split if it doesn't divide the dataset
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # calculate the information gain from this split
            gain = information_gain(true_rows, false_rows, current_uncertainty)

            # update best gain and question
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [ ]:
class Leaf:
    """A Leaf node classifies data."""

    def __init__(self, rows):
        self.predictions = class_counts(rows)
        self.label = rows[0][-1]

In [ ]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question and the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [ ]:
def build_dt(rows):
    """Build a decision tree using recursion"""

    # try split the dataset
    # calculate the information gain
    # and return the question that produces the highest gain
    gain, question = find_best_split(rows)

    # base case - no further information gain
    # return a leaf
    if gain == 0:
        return Leaf(rows)

    # if we reach here we have found a useful feature / value
    # to partition on
    true_rows, false_rows = partition_at_node(rows, question)

    # recursively build the true branch
    true_branch = build_dt(true_rows)

    # recursively build the false branch
    false_branch = build_dt(false_rows)

    # return a decision node
    # this records the best feature / value to ask at this point,
    # as well as the branches to follow
    # depending on the answer
    return Decision_Node(question, true_branch, false_branch)

In [ ]:
def classify(row, node):
    """Recursively move through the decision tree nodes until a leaf is reached"""

    # base case - return label of leaf
    if isinstance(node, Leaf):
        return node.label

    # determine whether to follow the true or false branch
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

### **Build Decision Tree and Output**

In [ ]:
# build the decision tree classifier with the training samples
dtc = build_dt(training)

In [ ]:
def display_tree(node, spacing=""):
    # base case - reached a leaf
    if isinstance(node, Leaf):
        print(spacing + "Prediction:", Classification(node.label).name)
        return

    # print the question at this node
    print(spacing + str(node.question))

    # recursively print the true branch
    print(spacing + '==> True:')
    display_tree(node.true_branch, spacing + "\t")

    # recursively print the false branch
    print(spacing + '==> False:')
    display_tree(node.false_branch, spacing + "\t")

display_tree(dtc)

variance >= 0.3223
==> True:
	curtosis >= -4.3839
	==> True:
		variance >= 1.594
		==> True:
			variance >= 2.0421
			==> True:
				Prediction: inauthentic
			==> False:
				curtosis >= -2.3386
				==> True:
					Prediction: inauthentic
				==> False:
					entropy >= -1.7207
					==> True:
						Prediction: authentic
					==> False:
						Prediction: inauthentic
		==> False:
			curtosis >= -2.2718
			==> True:
				entropy >= 0.23447
				==> True:
					curtosis >= 0.46497
					==> True:
						entropy >= 1.1008
						==> True:
							Prediction: authentic
						==> False:
							Prediction: inauthentic
					==> False:
						Prediction: authentic
				==> False:
					Prediction: inauthentic
			==> False:
				skewness >= 7.7763
				==> True:
					Prediction: inauthentic
				==> False:
					Prediction: authentic
	==> False:
		skewness >= 9.1814
		==> True:
			Prediction: inauthentic
		==> False:
			Prediction: authentic
==> False:
	skewness >= 7.0521
	==> True:
		variance >= -2.7143
		==> Tr

### **Results**

In [ ]:
# display the percentage of each class/label in the training data used

print('Percentage of \'authentic\' samples in training set: {0}%'
  .format(class_counts(training)['authentic'] / len(training)))
print('Percentage of \'inauthentic\' samples in training set: {0}%\n'
  .format(class_counts(training)['inauthentic'] / len(training)))

# print the actual and predicted label for the testing samples
# print an accuracy score by calculating num_correct / total num_samples

num_correct = 0
num_samples = len(testing)

for sample in testing:
    if sample[-1] == classify(sample, dtc):
      # correct predicition
      num_correct += 1
      print('Actual: {0}, Predicted: {1}'.format(sample[-1], classify(sample, dtc)))
    else:
      # incorrect - output sample for investigation
      print('Actual: {0}, Predicted: {1} {2}'
        .format(sample[-1], classify(sample, dtc), sample))

    
print('=======================')
print('Accuracy: {0}%'.format(num_correct / num_samples))

Percentage of 'authentic' samples in training set: 0.43624772313296906%
Percentage of 'inauthentic' samples in training set: 0.563752276867031%

Actual: 1, Predicted: 1
Actual: 1, Predicted: 1
Actual: 1, Predicted: 1
Actual: 1, Predicted: 1
Actual: 0, Predicted: 0
Actual: 0, Predicted: 0
Actual: 1, Predicted: 1
Actual: 1, Predicted: 1
Actual: 1, Predicted: 1
Actual: 0, Predicted: 0
Actual: 0, Predicted: 0
Actual: 0, Predicted: 0
Actual: 0, Predicted: 0
Actual: 0, Predicted: 0
Actual: 1, Predicted: 1
Actual: 1, Predicted: 1
Actual: 1, Predicted: 1
Actual: 1, Predicted: 1
Actual: 0, Predicted: 0
Actual: 1, Predicted: 1
Actual: 1, Predicted: 0 [0.37637, -0.82358, 0.78543, 0.74524, '1']
Actual: 0, Predicted: 1 [1.4806, 7.6377, -2.7876, -1.0341, '0']
Actual: 1, Predicted: 1
Actual: 0, Predicted: 0
Actual: 1, Predicted: 1
Actual: 0, Predicted: 0
Actual: 0, Predicted: 0
Actual: 0, Predicted: 0
Actual: 0, Predicted: 0
Actual: 0, Predicted: 0
Actual: 0, Predicted: 0
Actual: 0, Predicted: 0
Actu